# 算法评估

## 算法评估的基础知识

### 为什么需要算法评估
- **避免过度拟合**：如果仅用训练数据来评估模型，模型可能会过度适应这些数据（即过度拟合），无法准确预测新的、未见过的数据。
- **模型泛化能力**：评估是为了测试模型对新数据的预测能力，即它的**泛化能力**。

### 算法评估基本方法
1. **分离训练和评估数据集**：确保评估数据集与训练数据集不同，以公正地评估模型性能。
2. **重新采样评估方法**：使用不同的数据集合方法来更准确地估计模型性能。

### 算法评估常用的评估方法

1. 留出法（Holdout Method）
2. **K折交叉验证（K-Fold Cross-Validation）(Most recommended)**
3. 留一法（Leave-One-Out Cross-Validation, LOOCV）
4. 重复随机子抽样（Repeated Random Sub-sampling）

#### 分离训练集和验证集（Holdout Method）
- 内容
  1. 数据集被划分为两个独立的部分：训练集和评估集（或测试集）。
  2. 通常采用67%训练集和33%测试集的比例。
  3. 训练集用于模型训练，测试集用于评估。

- 根本差异
  1. 最简单和直接的方法。
  2. 对于大量数据效果好。
  3. 可能不足以评估模型的所有方面，特别是如果数据集不够大或不够代表性。

In [15]:
# 留出法(2:1分隔)
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

filename = 'pima_data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(filename, names = names)

array = data.values
X = array[:, 0:8]
Y = array[:, 8]

test_size = 0.33
seed = 4 # random number generator seed, so that we can reproduce the same results next time we run the code
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = test_size, random_state = seed)

model = LogisticRegression(multi_class = 'multinomial', max_iter = 2000) 
model.fit(X_train, Y_train)
result = model.score(X_test, Y_test)

print('Accuracy: %.3f%%' % (result * 100.0)) # 百分号后保留三位小数

Accuracy: 80.315%


#### K折交叉验证
- 内容
  1. 将数据集划分为K个大小相同的子集。
  2. 每次用K-1个子集训练模型，剩下的一个子集用于测试。
  3. 这个过程重复K次，每次选择不同的子集作为测试集。

- 根本差异
  1. 提供了更全面的方式来评估模型，因为每个数据点都有机会被用作测试集。
  2. 对于数据量较少的情况更有效。
  3. 计算成本高于简单的留出法。

In [13]:
# K折交叉验证(Most recommended)
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score # 导入交叉验证的评估指标
from sklearn.linear_model import LogisticRegression

filename = 'pima_data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(filename, names = names)

array = data.values
X = array[:, 0:8]
Y = array[:, 8]

num_folds = 10 # K通常取3、5、10
seed = 7
kfold = KFold(n_splits = num_folds, random_state = seed, shuffle = True)
model = LogisticRegression(multi_class = 'multinomial', max_iter = 2000) 
result = cross_val_score(model, X, Y, cv = kfold)

print('Accuracy: %.3f%% (%.3f%%)' % (result.mean() * 100.0, result.std() * 100.0)) # 平均值和标准差

Accuracy: 77.346% (4.900%)


#### 弃一交叉验证（Leave-One-Out Cross-Validation, LOOCV）
- 内容
  1. 是K折交叉验证的极端情况，K等于样本数。
  2. 每次只留一个样本作为测试集，其余作为训练集。
  3. 这个过程重复N次（N为样本数量）。

- 根本差异
  1. 提供非常精确的评估，因为几乎所有样本都用于训练。
  2. 没有随机性，因此结果非常稳定可靠。
  3. 计算成本非常高，尤其是在大型数据集上。

In [12]:
# 弃一交叉验证
from pandas import read_csv
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

filename = 'pima_data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(filename, names = names)

array = data.values
X = array[:, 0:8]
Y = array[:, 8]

loocv = LeaveOneOut()
model = LogisticRegression(multi_class = 'multinomial', max_iter = 2000) 
result = cross_val_score(model, X, Y, cv = loocv)

print('Accuracy: %.3f%% (%.3f%%)' % (result.mean() * 100.0, result.std() * 100.0))

Accuracy: 77.734% (41.603%)


#### 重复随机子抽样（Repeated Random Sub-sampling）
- 内容
  1. 多次随机将数据集分成训练集和测试集。
  2. 与留出法类似，但重复进行多次，每次分割都是随机的。

- 根本差异
  1. 提供了更多的数据多样性和重复性，有助于更准确地评估模型。
  2. 但与K折交叉验证相比，可能包含更多随机性和变异性。

In [11]:
# 重复随机子抽样
from pandas import read_csv
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

filename = 'pima_data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(filename, names = names)

array = data.values # 读取数据
X = array[:, 0:8]
Y = array[:, 8]

n_splits = 10
test_size = 0.33
seed = 7
kfold = ShuffleSplit(n_splits = n_splits, test_size = test_size, random_state = seed)
model = LogisticRegression(multi_class = 'multinomial', max_iter = 2000) 
result = cross_val_score(model, X, Y, cv = kfold)

print('Accuracy: %.3f%% (%.3f%%)' % (result.mean() * 100.0, result.std() * 100.0))

Accuracy: 76.378% (2.316%)


## 评估分类算法的方法

对于分类问题，评估分类算法的方法包括以下四种：
1. 分类准确度（无代码实现，不常用）
2. 对数损失函数
3. AUC图
4. 混淆矩阵

### 分类准确度
- 内容
  1. 定义：正确分类的样本数除以总样本数。
  2. 应用场景：最直观、最常用的评估指标，尤其是在类别分布均衡的情况下。

- 优缺点
  1. 优点：易于理解和计算。
  2. 缺点：在不平衡的数据集中可能产生误导。例如，在一个大多数样本属于同一类别的数据集中，模型可能仅仅通过预测这个主要类别就能获得很高的准确度，但这并不能全面反映模型的性能。

### 对数损失函数
- 内容
  1. 定义：在逻辑回归中，对数损失函数（或称为逻辑损失）是基于模型预测概率的负似然函数的平均值。
  2. 计算方式：考虑了模型预测的概率与实际标签的差异。

- 优缺点
  1. 优点：对模型预测的不确定性进行量化。不仅关注分类的正确与否，还关注预测的置信度。
  2. 缺点：更复杂，对于非专业人士来说可能难以直观理解。

In [16]:
# 对数损失函数
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

filename = 'pima_data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(filename, names = names)

array = data.values
X = array[:, 0:8]
Y = array[:, 8]

num_folds = 10
seed = 7
kfold = KFold(n_splits = num_folds, random_state = seed, shuffle = True)
model = LogisticRegression(multi_class = 'multinomial', max_iter = 2000)
scoring = 'neg_log_loss' # 对数损失函数
result = cross_val_score(model, X, Y, cv = kfold, scoring = scoring)

print('LogLoss: %.3f (%.3f)' % (result.mean(), result.std()))

LogLoss: -0.486 (0.057)


### AUC图

#### ROC曲线（Receiver Operating Characteristic Curve）
- **定义**：ROC曲线展示了分类器在所有可能的阈值下的真正类率（召回率）和假正类率（1-特异性）之间的关系。
- **敏感性和特异性**：
  - **敏感性**（又称召回率）：真正类（TP）相对于所有实际正类（TP + FN）的比率，高召回率意味着较少的正例被错误地划分为负例。
  - **特异性**：真负类（TN）相对于所有实际负类（TN + FP）的比率，高特异性意味着较少的负例被错误地划分为正例。
- **曲线特点**：理想的分类器的ROC曲线会尽可能靠近左上角，这意味着分类器在保持低假正类率的同时能达到高真正类率。

#### AUC（Area Under Curve）
- **定义**：AUC是ROC曲线下的面积，反映分类器区分正负类的能力。
- **值域**：AUC的值介于0.5（无区分能力）到1.0（完美区分）之间。
- **优点**：AUC作为单一数字，可以简便地比较不同模型的性能。

#### AUC图在不平衡数据集上的优势
- 在不平衡的数据集上，单纯的准确率（Accuracy）可能会误导。比如，在正样本占90%，负样本占10%的情况下，一个简单的预测所有结果为正样本的模型就能达到90%的准确率，但这并不代表模型有良好的分类能力。
- AUC作为一个综合指标，考虑了敏感性和特异性，因此更能反映模型在不平衡数据集上的表现。

#### AUC图的实际应用
在实际应用中，AUC和ROC曲线提供了一种评估分类器在不同阈值下平衡敏感性和特异性的能力的方法。通过分析ROC曲线和计算AUC值，我们可以更全面地理解分类器的性能，特别是在处理不平衡数据集时。

In [17]:
# AUC图
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve # ROC曲线
from sklearn.metrics import roc_auc_score # ROC曲线下的面积

filename = 'pima_data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(filename, names = names)

array = data.values
X = array[:, 0:8]
Y = array[:, 8]

num_folds = 10
seed = 7
kfold = KFold(n_splits = num_folds, random_state = seed, shuffle = True)
model = LogisticRegression(multi_class = 'multinomial', max_iter = 2000)
scoring = 'roc_auc' # AUC图
result = cross_val_score(model, X, Y, cv = kfold, scoring = scoring)

print('AUC: %.3f (%.3f)' % (result.mean(), result.std()))

AUC: 0.830 (0.046)


### 混淆矩阵（Confusion Matrix）

#### 混淆矩阵定义
- **混淆矩阵**是一个表格，用于显示分类模型在每个类别上的性能。
- 它将每个实例的**真实类别**与模型**预测的类别**进行对比。

#### 混淆矩阵结构（与ROC相关概念很相似）
- **列**：表示**预测**类别。列的总数是被预测为该类别的实例数。
- **行**：表示**实际**类别。行的总数是每个实际类别的实例数。
- 具体元素：显示为每个真实类别被预测为每个类别的次数。

例如，对于一个二分类问题，混淆矩阵可能如下所示：

| 真实\预测 | 正类 | 负类 |
|---------|----|----|
| 正类    | TP | FN |
| 负类    | FP | TN |

其中：
- TP（真正类）：正确地预测为正类。
- FN（假负类）：错误地预测为负类（实际为正类）。
- FP（假正类）：错误地预测为正类（实际为负类）。
- TN（真负类）：正确地预测为负类。

#### 分类报告

1. 精确率（Precision）
   - **定义**：在所有被预测为正类的实例中，实际为正类的比例。
   - **公式**：P = TP / (TP + FP)

2. 召回率（Recall）
   - **定义**：在所有实际为正类的实例中，被正确预测为正类的比例。
   - **公式**：R = TP / (TP + FN)

3. F1分数（F1 Score）
   - **定义**：精确率和召回率的调和平均值。
   - **重要性**：F1分数更加均衡地考虑了精确率和召回率，特别适合于不平衡数据集。
   - **公式**：F1 = 2 * (Precision * Recall) / (Precision + Recall)

#### 混淆矩阵重要性
- **混淆矩阵**提供了模型性能的详细视图，包括了各种类型的错误。
- **分类报告**提供了精确率、召回率和F1分数，这些都是评估模型性能的关键指标。

#### 混淆矩阵应用
- 在实际应用中，混淆矩阵和分类报告可以帮助我们识别模型的弱点，如是否存在某个类别的过度预测或者某类错误特别频繁等。
- 对于不平衡数据集，仅仅看准确度是不够的。F1分数、精确率和召回率提供了更全面的性能评估。

In [19]:
# 混淆矩阵
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix # 混淆矩阵
from sklearn.metrics import classification_report # 分类报告

filename = 'pima_data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = read_csv(filename, names = names)

array = data.values
X = array[:, 0:8]
Y = array[:, 8]
test_size = 0.33
seed = 4
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = test_size, random_state = seed)
model = LogisticRegression(multi_class = 'multinomial', max_iter = 2000)
model.fit(X_train, Y_train)
predicted = model.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
report = classification_report(Y_test, predicted)

print("Confusion matrix: ")
print(matrix)

print("Classification report: ")
print(report)

Confusion matrix: 
[[149  22]
 [ 28  55]]
Classification report: 
              precision    recall  f1-score   support

         0.0       0.84      0.87      0.86       171
         1.0       0.71      0.66      0.69        83

    accuracy                           0.80       254
   macro avg       0.78      0.77      0.77       254
weighted avg       0.80      0.80      0.80       254



## 评估回归算法的方法

评估回归算法有三种主要的评估指标：
1. 平均绝对误差（MAE）
2. 均方误差（MSE）
3. 决定系数（$ R^2 $）。

## 三种指标的总结
- **MAE** 提供了一个直接的平均误差度量。
- **MSE** 更强调大的误差。
- **$ R^2 $** 表明模型对数据变异的解释程度。

在实际应用中，根据具体问题的性质和需求选择最合适的评估指标。例如，如果你特别关心异常值对模型的影响，MSE可能是一个更好的选择。相反，如果你需要一个对异常值不太敏感的度量，那么MAE可能更合适。$ R^2 $ 对于评估模型的解释能力很有用，但它不应该是唯一的参考指标。

### 1. 平均绝对误差（MAE）

#### MAE定义
平均绝对误差是预测值与实际值之差的绝对值的平均。公式为：

$$ \text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i| $$

其中，$ y_i $ 是实际值，$ \hat{y}_i $ 是预测值，$ n $ 是样本数量。

#### MAE特点
- **非负值**：MAE永远是非负的，值越小表示模型越好。
- **易于理解**：直接反映了预测值和实际值之间的平均差距。
- **不敏感于异常值**：与平方误差相比，绝对误差不会放大异常值的影响。

In [6]:
# MAE
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

filename = 'housing.csv'
names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM',
         'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
data = read_csv(filename, delim_whitespace=True, names=names)
array = data.values
X = array[:, 0:13]
Y = array[:, 13]

kfold = KFold(n_splits = 10, random_state = 7, shuffle = True)
model = LinearRegression()
result = cross_val_score(model, X, Y, cv = kfold, scoring = 'neg_mean_absolute_error')

print("MAE: %.3f (%.3f)" % (-result.mean(), result.std()))

MAE: 3.387 (0.667)


### 2. 均方误差（MSE）

#### MSE定义
均方误差是预测值与实际值之差的平方的平均。公式为：

$$ \text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 $$

#### MSE特点
- **非负值**：MSE永远是非负的，值越小表示模型越好。
- **惩罚异常值**：由于平方项，MSE对异常值更敏感，即异常值会对MSE产生较大的影响。
- **单位问题**：MSE的单位是原始数据单位的平方，这可能使得解释变得复杂。


In [7]:
# MSE
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

filename = 'housing.csv'
names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM',
         'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
data = read_csv(filename, delim_whitespace=True, names=names)
array = data.values
X = array[:, 0:13]
Y = array[:, 13]
kfold = KFold(n_splits=10, random_state=7, shuffle=True)
model = LinearRegression()
result = cross_val_score(model, X, Y, cv=kfold, scoring='neg_mean_squared_error')

print("MSE: %.3f (%.3f)" % (-result.mean(), result.std()))

MSE: 23.747 (11.143)


### 3. 决定系数（$ R^2 $）

#### $ R^2 $定义
决定系数，也称为拟合优度，反映了模型对数据的拟合程度。公式为：

$$ R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2} $$

其中，$ \bar{y} $ 是实际值的平均值。

#### $ R^2 $特点
- **取值范围**：$ R^2 $ 的值在0到1之间，1表示完美拟合，0表示模型无法解释数据的变异。
- **解释性**：$ R^2 $ 易于理解，它提供了模型解释数据变异的能力。
- **局限性**：$ R^2 $ 值高并不意味着预测精度高，尤其是在非线性关系的情况下。

In [8]:
# R^2
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

filename = 'housing.csv'
names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM',
         'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
data = read_csv(filename, delim_whitespace=True, names=names)
array = data.values
X = array[:, 0:13]
Y = array[:, 13]

kfold = KFold(n_splits=10, random_state=7, shuffle=True)
model = LinearRegression()
result = cross_val_score(model, X, Y, cv=kfold, scoring='r2')

print("R^2: %.3f (%.3f)" % (result.mean(), result.std()))

R^2: 0.718 (0.099)
